In [1]:
import os
import re 
import glob
import openpyxl

In [ ]:
# C:\Users\kazit\Downloads\parent  

In [7]:
# Function to create an Excel file with subfolder names, MAIN, and ASIN columns
def create_excel_file(parent_folder):
    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    worksheet.title = "Subfolders"

    # Add headers
    worksheet.cell(row=1, column=1, value="Subfolder Name")
    worksheet.cell(row=1, column=2, value="MAIN")
    worksheet.cell(row=1, column=3, value="ASIN")

    # List all subfolders and add them to the Excel file
    subfolders = [f.name for f in os.scandir(parent_folder) if f.is_dir()]
    for i, subfolder in enumerate(subfolders, start=2):
        worksheet.cell(row=i, column=1, value=subfolder)

    # Save the Excel file
    excel_file = os.path.join(parent_folder, "subfolders.xlsx")
    workbook.save(excel_file)

    return excel_file

# Input the parent folder path
parent_folder = input("Enter the path of the parent folder: ")

# Create the Excel file with subfolder names
excel_file = create_excel_file(parent_folder)

print(f"Excel file 'subfolders.xlsx' created in '{parent_folder}'")
print("Please fill in the ASIN and MAIN columns as needed.")
print("Once you've filled in the Excel file, run the utility again to rename the files.")


Enter the path of the parent folder: C:\Users\kazit\Downloads\parent
Excel file 'subfolders.xlsx' created in 'C:\Users\kazit\Downloads\parent'
Please fill in the ASIN and MAIN columns as needed.
Once you've filled in the Excel file, run the utility again to rename the files.


In [8]:
# Function to read the Excel file and return subfolder data
def read_excel_file(excel_file):
    subfolder_data = []

    if os.path.exists(excel_file):
        workbook = openpyxl.load_workbook(excel_file)
        worksheet = workbook.active

        for row in worksheet.iter_rows(min_row=2, values_only=True):
            subfolder_data.append(row)

    return subfolder_data

# Function to extract numerical values from folder names
def extract_numerical_value(folder_name):
    match = re.search(r'\d+', folder_name)
    return int(match.group()) if match else 0

# Function to rename files within a subfolder based on subfolder data
def rename_files(subfolder_path, main_file_name, asin_number):
    main_exists = False

    for filename in os.listdir(subfolder_path):
        if main_file_name and filename.upper() == main_file_name.upper():
            main_exists = True
            continue  # Skip renaming the main file if it exists

        base, ext = os.path.splitext(filename)
        if main_file_name and "MAIN" in main_file_name.upper():
            new_filename = f"{asin_number}.MAIN{ext}"
        elif main_file_name and "_1" in main_file_name:
            new_filename = f"{asin_number}.MAIN{ext}"
        else:
            if main_exists:
                new_filename = f"{asin_number}.PT{extract_numerical_value(base):02d}{ext}"
            else:
                new_filename = f"{asin_number}.MAIN{ext}"
        
        src_file_path = os.path.join(subfolder_path, filename)
        dst_file_path = os.path.join(subfolder_path, new_filename)

        if not os.path.exists(dst_file_path):
            os.rename(src_file_path, dst_file_path)
            print(f"Renamed: {filename} -> {new_filename}")
        else:
            print(f"File '{new_filename}' already exists in '{subfolder_path}', skipping.")

# Input the parent folder path
parent_folder = input("Enter the path of the parent folder: ")

excel_file = os.path.join(parent_folder, "subfolders.xlsx")

subfolder_data = read_excel_file(excel_file)

if subfolder_data:
    for subfolder_name, main_file_name, asin_number in subfolder_data:
        subfolder_path = os.path.join(parent_folder, subfolder_name)
        if os.path.exists(subfolder_path) and os.path.isdir(subfolder_path):
            rename_files(subfolder_path, main_file_name, asin_number)
        else:
            print(f"Subfolder '{subfolder_name}' not found in '{parent_folder}'.")

else:
    print(f"No subfolder data found in '{excel_file}'.")


Enter the path of the parent folder: C:\Users\kazit\Downloads\parent
Renamed: 235.main_MAIN_1.png -> 234.MAIN.png
File '234.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (2)', skipping.
File '234.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (2)', skipping.
File '234.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (2)', skipping.
Renamed: 235.main_1.png -> 235.MAIN.png
File '235.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (3)', skipping.
File '235.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (3)', skipping.
File '235.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (3)', skipping.
Renamed: 235.main_MAIN_1.png -> 236.MAIN.png
File '236.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (4)', skipping.
File '236.MAIN.png' already exists in 'C:\Users\kazit\Downloads\parent\New folder (4)', skipping.
File '236.MAIN.pn

In [9]:
# Function to read the Excel file and return subfolder data
def read_excel_file(excel_file):
    subfolder_data = []

    if os.path.exists(excel_file):
        workbook = openpyxl.load_workbook(excel_file)
        worksheet = workbook.active

        for row in worksheet.iter_rows(min_row=2, values_only=True):
            subfolder_data.append(row)

    return subfolder_data

# Function to rename files within a subfolder based on subfolder data
def rename_files(subfolder_path, main_file_name, asin_number):
    # Create a list of files in the subfolder excluding the main file
    files_to_rename = []
    for filename in os.listdir(subfolder_path):
        if filename != main_file_name and "main" not in filename.lower():
            files_to_rename.append(filename)

    # Sort the list of files and rename them in the order of "ASIN_Number.PT01, PT02, PT03, ..."
    pt_number = 1
    for filename in sorted(files_to_rename):
        base, ext = os.path.splitext(filename)
        new_filename = f"{asin_number}.PT{pt_number:02d}{ext}"
        
        src_file_path = os.path.join(subfolder_path, filename)
        dst_file_path = os.path.join(subfolder_path, new_filename)
        
        os.rename(src_file_path, dst_file_path)
        print(f"Renamed: {filename} -> {new_filename}")
        
        pt_number += 1

# Input the parent folder path
parent_folder = input("Enter the path of the parent folder: ")

excel_file = os.path.join(parent_folder, "subfolders.xlsx")

subfolder_data = read_excel_file(excel_file)

if subfolder_data:
    for subfolder_name, main_file_name, asin_number in subfolder_data:
        subfolder_path = os.path.join(parent_folder, subfolder_name)
        if os.path.exists(subfolder_path) and os.path.isdir(subfolder_path):
            rename_files(subfolder_path, main_file_name, asin_number)
        else:
            print(f"Subfolder '{subfolder_name}' not found in '{parent_folder}'.")

else:
    print(f"No subfolder data found in '{excel_file}'.")


Enter the path of the parent folder: C:\Users\kazit\Downloads\parent
Renamed: image_1.png -> 234.PT01.png
Renamed: image_2.png -> 234.PT02.png
Renamed: image_3.png -> 234.PT03.png
Renamed: image_1.png -> 235.PT01.png
Renamed: image_2.png -> 235.PT02.png
Renamed: image_3.png -> 235.PT03.png
Renamed: image_1.png -> 236.PT01.png
Renamed: image_2.png -> 236.PT02.png
Renamed: image_3.png -> 236.PT03.png
Renamed: image_1.png -> 237.PT01.png
Renamed: image_2.png -> 237.PT02.png
Renamed: image_3.png -> 237.PT03.png
Renamed: image_1.png -> 238.PT01.png
Renamed: image_2.png -> 238.PT02.png
Renamed: image_3.png -> 238.PT03.png
